In [1]:
import datetime
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt



In [2]:
#Created a 07/23/2021 
#Last opened 
print(f'Last edited: {datetime.datetime.now()}')

Last edited: 2022-12-15 11:45:15.412438


In [3]:
# load constraints from the data and save all the means and 2nd moments, then save all the required points 
## saving Data of IGF
filepath  = 'Data/IGF/EXP129_RAW.xlsx'
xls = pd.ExcelFile(filepath)
Data_Dict = defaultdict(lambda: 'Not present')
Sheet_labels = ['Sheet2' ,'0pM','10pM','15pm','20pM','25pM', '50pM']
# conc
nSheets = len(Sheet_labels)
for i in range(nSheets): 
    sl = Sheet_labels[i]
    df = pd.read_excel(xls, sl ,header=None)
    data = df.to_numpy()
    Data_Dict[sl] = data
    print(f'saved {sl} to dictionary')


saved Sheet2 to dictionary
saved 0pM to dictionary
saved 10pM to dictionary
saved 15pm to dictionary
saved 20pM to dictionary
saved 25pM to dictionary
saved 50pM to dictionary


In [12]:
print(Data_Dict.keys())
Data_Dict['10pM'].shape

dict_keys(['Sheet2', '0pM', '10pM', '15pm', '20pM', '25pM', '50pM'])


(61, 302)

### Getting the scaling factor 

In [14]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1] # defining the background noise array -- it is column indx 1 because 0 is the minute number
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))
conc_labels = ['10pM','15pm','20pM','50pM']
# conc
bgg = 0     
ncc = 0 # number of cells 
# Means_dict = defaultdict(lambda: 'Not present')
# Var_dict = defaultdict(lambda:'Not present')
# SecondMoment_dict = defaultdict(lambda:'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    print('shape of measurement matrix', matrix.shape)
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
    print('shape of bg matrix', bg_matrix.shape)
    
#     print(matrix.shape)
    ## subtract 
    data = matrix - bg_matrix
    
    print( 'shape of bg substracted data' ,data.shape)
    bgg += np.sum(data[0,:]) #  running sum of flourescence units at t = 0 -- cells have not responded yet 
    ncc += len(data[0,:]) # running sum of cell number 
    print(len(data[0,:]) )
    
flour_noligand = bgg/ncc #estimate of flourescence units at no ligand -- when cells haven't responded to IGF yet 
print(flour_noligand)

10pM
shape of measurement matrix (61, 301)
shape of bg matrix (61, 301)
shape of bg substracted data (61, 301)
301
15pm
shape of measurement matrix (61, 300)
shape of bg matrix (61, 300)
shape of bg substracted data (61, 300)
300
20pM
shape of measurement matrix (61, 300)
shape of bg matrix (61, 300)
shape of bg substracted data (61, 300)
300
50pM
shape of measurement matrix (61, 300)
shape of bg matrix (61, 300)
shape of bg substracted data (61, 300)
300
22530.951948376358


In [16]:
n_foxO = 710*2/3 
au_foxO = flour_noligand
# scale factor 
sc = n_foxO/au_foxO
print(f'sc = {sc}')

sc = 0.0210081373577934


### Removing the background and getting the shift

In [17]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

conc_labels = ['10pM','15pm','20pM','50pM']
# conc
bgg = 0 
ncc = 0

Means_dict = defaultdict(lambda: 'Not present')
# Var_dict = defaultdict(lambda:'Not present')
# SecondMoment_dict = defaultdict(lambda:'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
#     print(matrix.shape)
    ## subtract 
    data = (matrix - bg_matrix)*sc
    print(data.shape)
#     if key 
#     bgg += np.sum(data[0,:])
#     ncc += len(data[0,:]) 
    # Means_dict[key] = np.mean(data, axis = 1)
#     SecondMoment_dict[key] = np.mean(data**2, axis = 1)
#     Var_dict[key] = np.var(data, axis = 1)
    
# bgg/ncc



10pM
(61, 301)
15pm
(61, 300)
20pM
(61, 300)
50pM
(61, 300)


In [8]:
## make means matrix 
# time_values = [0,6,12,24,45,60,90]
means_matrix = np.zeros((len(conc_labels), 61))
i=0
for key in conc_labels:
    means_matrix[i,:] = Means_dict[key]
    i+=1 
np.shape(means_matrix)
    
    

(4, 61)

In [11]:
data_shift_dict = defaultdict(lambda: 'not present')
data_shift = (np.average(means_matrix[:,0]) - means_matrix[:,0])
for i in range(np.shape(means_matrix)[0]):
    
    means_matrix[i] = means_matrix[i] + data_shift[i]
    

### Saving the data with bg removal, shift added and scaling .

In [20]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

conc_labels = ['10pM','15pm','20pM','50pM']
# conc
bgg = 0 
ncc = 0
Means_dict = defaultdict(lambda: 'Not present')
Var_dict = defaultdict(lambda:'Not present')
SecondMoment_dict = defaultdict(lambda:'Not present')
log_dict  = defaultdict(lambda: 'Not present')
scale_Data = 1
i=0
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
    
#     print((matrix - bg_matrix)*sc )
    
    data =( ((matrix - bg_matrix)*sc ) + data_shift[i])/scale_Data
#     data
    print(data.shape)
    Means_dict[key] = np.mean(data, axis = 1)
#     print(np.mean(data, axis = 1).shape)
#     print('log' ,np.mean(np.log(data), axis = 1).shape)
    SecondMoment_dict[key] = np.mean(data**2, axis = 1)
    Var_dict[key] = np.var(data, axis = 1)
    log_dict[key] = np.mean(np.log(data), axis = 1)
    
    
    i+=1
    
# bgg/ncc



10pM
(61, 301)
15pm
(61, 300)
20pM
(61, 300)
50pM
(61, 300)


In [21]:
Means_dict

defaultdict(<function __main__.<lambda>()>,
            {'10pM': array([473.3266724 , 431.45420641, 454.10860632, 455.47604273,
                    454.92797602, 448.15236735, 445.07934727, 440.7862995 ,
                    436.5247793 , 432.34405597, 428.22913836, 424.4263926 ,
                    420.49208286, 416.70427033, 414.2070812 , 410.89845591,
                    407.75641816, 405.78816368, 403.35876034, 401.27088371,
                    399.71709395, 398.37611314, 397.68485608, 397.08334355,
                    396.85881961, 396.64753358, 395.77463082, 395.87751625,
                    395.90583327, 396.31201967, 396.37161368, 396.98211085,
                    397.01623756, 397.07629641, 397.47630111, 398.39882775,
                    397.89450473, 398.2314934 , 399.0371087 , 398.56416458,
                    398.63094114, 399.41960965, 400.06880018, 400.3803362 ,
                    400.80919813, 401.22418353, 401.71026576, 402.39722417,
                    403.17711184, 40

In [22]:
SecondMoment_dict

defaultdict(<function __main__.<lambda>()>,
            {'10pM': array([228865.06950352, 190155.32385097, 210509.3377612 , 211909.59371188,
                    211701.23741619, 205625.19415699, 202972.37919141, 199194.06502796,
                    195448.77434065, 191906.59619268, 188422.51076412, 185264.33598773,
                    181942.27357214, 178924.87649242, 176863.67555538, 174203.73968426,
                    171679.47263427, 170197.23261482, 168307.2773056 , 166707.68267513,
                    165509.61633014, 164399.83562526, 163904.56898962, 163440.44822613,
                    163242.63675272, 163208.33217853, 162364.41026372, 162379.01395877,
                    162341.99271116, 162664.89975456, 162740.57904566, 163274.69538166,
                    163251.57176754, 163439.92985405, 163727.82711309, 164505.19960069,
                    164004.74062947, 164359.91981125, 165020.30231342, 164582.99086068,
                    164567.78491319, 165238.67889436, 165686.4273036

### Getting the constraints and saving them 

In [23]:
time_values = [0,6,12,24,45,60,90]
times_constrained = np.array(time_values) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)

In [14]:
times_constrained

array([ 0,  6, 12, 24, 45, 60, 90])

In [15]:
idx_list

array([ 0,  2,  4,  8, 15, 20, 30])

In [24]:
# constraints_means_dict = defaultdict(lambda: 'not present')
conc_list = [10,15,20,50]
conc_labels = ['10pM','15pm','20pM','50pM']
labels_arr_means = []
labels_arr_sm = []
labels_arr_log = []
cons_arr = np.zeros(len(conc_list)* len(time_values)*2)
k=0
titles_list = []
means_list = []
sm_list = []
log_list = []
for i in range(len(conc_list)):
    for j in range(len(time_values)):
        title = f'{conc_list[i]}_pM_{time_values[j]}_min'
        # save the means
        key = conc_labels[i]
        means_list.append(Means_dict[key][idx_list[j]])
        # save the second moments 
        sm_list.append(SecondMoment_dict[key][idx_list[j]])
        ### 
        log_list.append(log_dict[key][idx_list[j]])
        # save the titles 
        labels_arr_means.append(f'Means_{title}')
        labels_arr_sm.append(f'sm_{title}')
        labels_arr_log.append(f'log_{title}')
        



In [27]:
log_list

[6.149198782866299,
 6.108245048965151,
 6.109049542470153,
 6.066413884420873,
 6.002727551696992,
 5.9728853980507015,
 5.9642980232151395,
 6.148452950275102,
 6.127032427513213,
 6.094487769304551,
 5.997529840695776,
 5.904326142306675,
 5.85977955295276,
 5.852184551283319,
 6.149523390949151,
 6.078590069850282,
 6.055443231584044,
 5.9242155284802225,
 5.774075550872303,
 5.733304738889447,
 5.727271249401523,
 6.14559706132277,
 6.0552089206030475,
 5.947980726817424,
 5.585587209730893,
 5.358005923074434,
 5.345941675182775,
 5.364040872071394]

In [24]:
means_arr = np.asarray(means_list)
sm_arr = np.asarray(sm_list)
log_arr = np.asarray(log_list)

## 
cons_arr = np.concatenate((means_arr, sm_arr), axis = 0 )
labels_arr = labels_arr_means + labels_arr_sm
### 


print(means_arr)
print(sm_arr)
print(labels_arr)
print(cons_arr)

[473.3266724  454.10860632 454.92797602 436.5247793  410.89845591
 399.71709395 396.37161368 473.3266724  463.36560404 448.82085023
 408.61048502 374.72214861 359.57389104 357.00817722 473.3266724
 440.84614128 431.31182823 380.0354967  330.08426843 317.88807432
 316.34335599 473.3266724  432.45472093 389.35206541 273.95826841
 219.50611675 216.90903078 221.13188648]
[228865.06950352 210509.3377612  211701.23741619 195448.77434065
 174203.73968426 165509.61633014 162740.57904566 229008.29654811
 219578.28377667 206260.52437945 171882.22597873 146135.87758281
 135316.89105136 133540.26892794 228658.59658047 198284.24352742
 190315.67890255 149111.42986571 114298.31209115 106619.13732642
 105796.70653268 230436.02119859 192391.25826074 156617.42018588
  79183.7396712   51405.61136711  50226.05458017  52304.21568455]
['Means_10_pM_0_min', 'Means_10_pM_6_min', 'Means_10_pM_12_min', 'Means_10_pM_24_min', 'Means_10_pM_45_min', 'Means_10_pM_60_min', 'Means_10_pM_90_min', 'Means_15_pM_0_min', 

In [28]:
means_arr = np.asarray(means_list)
# sm_arr = np.asarray(sm_list)
log_arr = np.asarray(log_list)

## 
cons_arr = np.concatenate((means_arr, log_arr), axis = 0 )
labels_arr = labels_arr_means + labels_arr_log


In [29]:
labels_arr

['Means_10_pM_0_min',
 'Means_10_pM_6_min',
 'Means_10_pM_12_min',
 'Means_10_pM_24_min',
 'Means_10_pM_45_min',
 'Means_10_pM_60_min',
 'Means_10_pM_90_min',
 'Means_15_pM_0_min',
 'Means_15_pM_6_min',
 'Means_15_pM_12_min',
 'Means_15_pM_24_min',
 'Means_15_pM_45_min',
 'Means_15_pM_60_min',
 'Means_15_pM_90_min',
 'Means_20_pM_0_min',
 'Means_20_pM_6_min',
 'Means_20_pM_12_min',
 'Means_20_pM_24_min',
 'Means_20_pM_45_min',
 'Means_20_pM_60_min',
 'Means_20_pM_90_min',
 'Means_50_pM_0_min',
 'Means_50_pM_6_min',
 'Means_50_pM_12_min',
 'Means_50_pM_24_min',
 'Means_50_pM_45_min',
 'Means_50_pM_60_min',
 'Means_50_pM_90_min',
 'log_10_pM_0_min',
 'log_10_pM_6_min',
 'log_10_pM_12_min',
 'log_10_pM_24_min',
 'log_10_pM_45_min',
 'log_10_pM_60_min',
 'log_10_pM_90_min',
 'log_15_pM_0_min',
 'log_15_pM_6_min',
 'log_15_pM_12_min',
 'log_15_pM_24_min',
 'log_15_pM_45_min',
 'log_15_pM_60_min',
 'log_15_pM_90_min',
 'log_20_pM_0_min',
 'log_20_pM_6_min',
 'log_20_pM_12_min',
 'log_20_pM_2

In [30]:
dictionary = {'labels': labels_arr, 'array':cons_arr}

In [35]:
4*7

28

In [36]:
np.save('/Users/hodaakl/Documents/GitHub/MaxEnt_FoxO/Arrays_for_max_ent/MuLnCons_28Conds_0119.npy', dictionary) 



In [65]:
np.save('/Volumes/hodaakl/A5MCMC_IGF_FoxO/Arrays_for_max_ent/Cons_1208_Zero2ndMoments.npy', dictionary) 



In [58]:
dictionary

{'labels': ['10_pM_0_min',
  '10_pM_6_min',
  '10_pM_12_min',
  '10_pM_24_min',
  '10_pM_45_min',
  '10_pM_60_min',
  '10_pM_90_min',
  '15_pM_0_min',
  '15_pM_6_min',
  '15_pM_12_min',
  '15_pM_24_min',
  '15_pM_45_min',
  '15_pM_60_min',
  '15_pM_90_min',
  '20_pM_0_min',
  '20_pM_6_min',
  '20_pM_12_min',
  '20_pM_24_min',
  '20_pM_45_min',
  '20_pM_60_min',
  '20_pM_90_min',
  '50_pM_0_min',
  '50_pM_6_min',
  '50_pM_12_min',
  '50_pM_24_min',
  '50_pM_45_min',
  '50_pM_60_min',
  '50_pM_90_min'],
 'array': array([473.3266724 , 454.10860632, 454.92797602, 436.5247793 ,
        410.89845591, 399.71709395, 396.37161368, 473.3266724 ,
        463.36560404, 448.82085023, 408.61048502, 374.72214861,
        359.57389104, 357.00817722, 473.3266724 , 440.84614128,
        431.31182823, 380.0354967 , 330.08426843, 317.88807432,
        316.34335599, 473.3266724 , 432.45472093, 389.35206541,
        273.95826841, 219.50611675, 216.90903078, 221.13188648])}

In [18]:
# n_foxO = 710*2/3 
# au_foxO = 22530.951948376358
# # scale factor 
# sc = n_foxO/au_foxO
# # sc

# # initialize data matrix that contains mean values 
# means_matrix = np.zeros((len(Means_dict),data.shape[0]))
# i=0
# for key in Means_dict: 
#     means_matrix[i,:] = Means_dict[key]* sc
#     i+=1 
    


In [33]:
# these have 61 points in each array, for 61 measurements 
SecondMoment_dict = dict(Means_dict)
np.save('Arrays_for_max_ent/SecondMoment_Dictionary.npy', SecondMoment_dict)#, allow_pickle=True) 

In [34]:
# these have 61 points in each array, for 61 measurements 
Var_dict = dict(Var_dict)
np.save('Arrays_for_max_ent/Variance_Dictionary.npy', Var_dict)#, allow_pickle=True) 

In [35]:
np.load('Arrays_for_max_ent/Means_Dictionary.npy', allow_pickle=True)

array({'0pM': array([25489.34666667, 22560.97666667, 24270.98333333, 24165.06      ,
       24214.46666667, 24113.4       , 24133.50333333, 24191.44333333,
       24239.72      , 24300.82      , 24307.35666667, 24378.20333333,
       24378.68      , 24454.94      , 24452.21666667, 24502.26      ,
       24531.81666667, 24562.89333333, 24566.34      , 24585.95      ,
       24631.36      , 24629.25333333, 24668.28      , 24675.84333333,
       24706.13666667, 24717.19666667, 24696.08666667, 24704.83333333,
       24721.64666667, 24773.01333333, 24815.79666667, 24802.21666667,
       24807.68      , 24816.63666667, 24859.97      , 24886.76333333,
       24925.70666667, 24942.10666667, 24960.27      , 24972.66      ,
       25040.71666667, 25031.67      , 25029.28      , 25051.52666667,
       25075.49666667, 25083.17      , 25111.35333333, 25124.02333333,
       25132.53333333, 25175.16333333, 25166.63666667, 25162.2       ,
       25206.18333333, 25228.42666667, 25229.37666667, 25244.85

In [7]:
# Making the constrained array 
# for all the concentrations except 0 pM
# 0, 6, 15, 21, 51, 126 min
times_constrained = np.array([0, 6, 12, 25, 45,60, 90]) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)

In [22]:
# Now making the constrained dictionary 

array([ 0,  2,  5,  7, 17, 42])

In [8]:
Constraints_Dict = defaultdict(lambda:'NotPresent')
Constraints_Dict['Mean']= {}
Constraints_Dict['Var'] = {}
Constraints_Dict['SecMoment'] = {}


Constraints_Dict['Times(min)'] = times_constrained

for i in range(len(Sheet_labels) -1): 
    conc = Sheet_labels[i+1]
    if conc == '15pm':
        con_key = '15pM'
    else:
        con_key = conc
    Constraints_Dict['Mean'][con_key] = Means_dict[conc][idx_list]
    Constraints_Dict['SecMoment'][con_key] = SecondMoment_dict[conc][idx_list]
    Constraints_Dict['Var'][con_key] = Var_dict[conc][idx_list]

In [6]:
Constraints_Dict

defaultdict(<function __main__.<lambda>()>,
            {'Mean': {'10pM': array([26629.13953488, 25580.74086379, 25612.27906977, 24624.97009967,
                     23366.93023256, 22810.94352159, 22608.14950166]),
              '15pM': array([26377.93333333, 25770.17333333, 25070.37      , 23045.02666667,
                     21393.71      , 20648.89666667, 20483.22      ]),
              '20pM': array([26263.96      , 24584.26      , 24122.95666667, 21570.86666667,
                     19154.94666667, 18550.65333333, 18433.57666667]),
              '25pM': array([25028.07      , 24102.68333333, 23689.69666667, 20595.28333333,
                     17104.38      , 16043.79666667, 15474.88666667]),
              '50pM': array([25722.34666667, 23643.21      , 21584.03333333, 15979.91333333,
                     13349.74666667, 13202.37666667, 13359.84      ]),
              '250pM': array([25364.01666667, 22210.54      , 17204.36      , 12114.49      ,
                     11753.6933333

In [9]:
times_constrained

array([ 0,  6, 12, 25, 45, 60, 90])

In [20]:
n_foxO = 710*2/3 
au_foxO = 25299.566666666666
# scale factor 
sc = n_foxO/au_foxO
sc

0.01870914785101721

In [22]:
LigandCon = [10,15,20,25,50,250]
nTcon = len(times_constrained)
nLcon = len(LigandCon)
nCons = nTcon*nLcon*2

Cons_means = []
Cons_sm    = []
Cons_names = []
# store means first 

for ligand in LigandCon: 
    key = f'{ligand}pM'
    ii = 0
    for timecon in times_constrained: 
        timestring = f'{timecon}min'
        title = f'{key}_{timestring}'
        Cons_names.append(title)
        Cons_means.append(Constraints_Dict['Mean'][key][ii])
        Cons_sm.append(Constraints_Dict['SecMoment'][key][ii])
        ii+=1 
        

# for key in Constraints_Dict['Mean']: 
#     Cons_Arr[ii:(ii+6)] = Constraints_Dict['Mean'][key]
#     ii+=6
Cons_means = np.asarray(Cons_means)*sc
Cons_sm = np.asarray(Cons_sm)*(sc**2)

In [23]:
con_dic = { 'labels' : [], 'array' : []}
i = 0 
for tit in Cons_names: 
    con_dic['labels'].append(f'mean_{tit}')
    con_dic['array'].append(Cons_means[i])
    i+=1 

    

i = 0 
for tit in Cons_names: 
    con_dic['labels'].append(f'sm_{tit}')
    con_dic['array'].append(Cons_sm[i])
    i+=1 


# 
con_dic['array']= np.asarray(con_dic['array'])
con_dic

{'labels': ['mean_10pM_0min',
  'mean_10pM_6min',
  'mean_10pM_12min',
  'mean_10pM_25min',
  'mean_10pM_45min',
  'mean_10pM_60min',
  'mean_10pM_90min',
  'mean_15pM_0min',
  'mean_15pM_6min',
  'mean_15pM_12min',
  'mean_15pM_25min',
  'mean_15pM_45min',
  'mean_15pM_60min',
  'mean_15pM_90min',
  'mean_20pM_0min',
  'mean_20pM_6min',
  'mean_20pM_12min',
  'mean_20pM_25min',
  'mean_20pM_45min',
  'mean_20pM_60min',
  'mean_20pM_90min',
  'mean_25pM_0min',
  'mean_25pM_6min',
  'mean_25pM_12min',
  'mean_25pM_25min',
  'mean_25pM_45min',
  'mean_25pM_60min',
  'mean_25pM_90min',
  'mean_50pM_0min',
  'mean_50pM_6min',
  'mean_50pM_12min',
  'mean_50pM_25min',
  'mean_50pM_45min',
  'mean_50pM_60min',
  'mean_50pM_90min',
  'mean_250pM_0min',
  'mean_250pM_6min',
  'mean_250pM_12min',
  'mean_250pM_25min',
  'mean_250pM_45min',
  'mean_250pM_60min',
  'mean_250pM_90min',
  'sm_10pM_0min',
  'sm_10pM_6min',
  'sm_10pM_12min',
  'sm_10pM_25min',
  'sm_10pM_45min',
  'sm_10pM_60min',
 

In [24]:
# Save that dictionary 

# dictionary = {'hello':'world'}
np.save('Cons_1117.npy', con_dic) 

# # Load
# read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()
# print(read_dictionary['hello']) # displays "world"




In [25]:
read_dictionary = np.load('Cons_1117.npy',allow_pickle='TRUE').item()

In [27]:
read_dictionary['array']

array([4.98208509e+02, 4.78593863e+02, 4.79183916e+02, 4.60712206e+02,
       4.37175353e+02, 4.26773315e+02, 4.22979212e+02, 4.93508655e+02,
       4.82137983e+02, 4.69045259e+02, 4.31152811e+02, 4.00258083e+02,
       3.86323261e+02, 3.83223591e+02, 4.91376311e+02, 4.59950555e+02,
       4.51319963e+02, 4.03572534e+02, 3.58372729e+02, 3.47066916e+02,
       3.44876511e+02, 4.68253862e+02, 4.50940666e+02, 4.43214037e+02,
       3.85320201e+02, 3.20008374e+02, 3.00165764e+02, 2.89521943e+02,
       4.81243187e+02, 4.42344312e+02, 4.03818871e+02, 2.98970561e+02,
       2.49762384e+02, 2.47005217e+02, 2.49951222e+02, 4.74539138e+02,
       4.15540277e+02, 3.21878915e+02, 2.26651785e+02, 2.19901586e+02,
       2.17709124e+02, 2.15389564e+02, 2.52040001e+05, 2.32458260e+05,
       2.33377969e+05, 2.16137920e+05, 1.95378270e+05, 1.86684626e+05,
       1.83376715e+05, 2.47492669e+05, 2.36319952e+05, 2.23826533e+05,
       1.89794603e+05, 1.64742474e+05, 1.54022961e+05, 1.51686731e+05,
      